In [9]:
import torch
from datasets import *
import os
import pyreadr
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Input, Dense, LSTM,Conv2D,Conv1D,Flatten,MaxPooling2D,GlobalAveragePooling2D,Dropout,Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import keras_nlp
import keras_nlp.layers

In [3]:
DATASET_FOLDER_PATH = "dataset/RData"

# faulty_train = torch.load(os.path.join(TORCH_DATASET_FOLDER_PATH, "TEP_Faulty_Training.torch"), weights_only= True)
# faulty_test = torch.load(os.path.join(TORCH_DATASET_FOLDER_PATH, "TEP_Faulty_Testing.torch"), weights_only= True)
# faultfree_train = torch.load(os.path.join(TORCH_DATASET_FOLDER_PATH, "TEP_FaultFree_Training.torch"), weights_only= True)
# faultfree_test = torch.load(os.path.join(TORCH_DATASET_FOLDER_PATH, "TEP_FaultFree_Testing.torch"), weights_only= True)
faulty_train_df = pyreadr.read_r(os.path.join(DATASET_FOLDER_PATH, "TEP_Faulty_Training.RData"))["faulty_training"]

faultfree_train_df = pyreadr.read_r(os.path.join(DATASET_FOLDER_PATH, "TEP_FaultFree_Training.RData"))["fault_free_training"]



In [4]:
df = pd.concat((faultfree_train_df, faulty_train_df))

In [31]:
df[df["faultNumber"] == 1].head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,1.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,1.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,1.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,1.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [5]:
# processed_data = df.loc[df["sample"] <= 40]
processed_data = df
processed_data = processed_data[processed_data["faultNumber"] != 3]
processed_data = processed_data[processed_data["faultNumber"] != 9]
processed_data = processed_data[processed_data["faultNumber"] != 15]

np_dataset = processed_data.to_numpy().reshape((len(pd.unique(processed_data["faultNumber"])),
                                                len(pd.unique(processed_data["simulationRun"])),
                                                len(pd.unique(processed_data["sample"])),
                                                55))
print(np_dataset.shape)
print(len(processed_data))
# processed_data.head()

(18, 500, 500, 55)
4500000


In [6]:
sample_count = 500
width = 20
stride = 10
lookahead = 5
start_idx_range = range(0, sample_count-width-lookahead, stride)

x_shape = list(np_dataset.shape)
x_shape[-2] = len(start_idx_range)
x_shape.insert(-1, width)
y_shape = list(x_shape)

x_shape[-1] = 52
y_shape = y_shape[:-1]
y_shape[-1] = 1

x_dataset = np.zeros(x_shape, dtype=np.float32)
y_dataset = np.zeros(y_shape, dtype=np.float32)
print(x_dataset.shape)
print(y_dataset.shape)
idx = 0

for start_idx in start_idx_range:
    x_dataset[:, :, idx, :, :] = np_dataset[:, :, start_idx:start_idx+width, 3:]
    y_dataset[:, :, idx, :] = np_dataset[:, :, start_idx+width+lookahead, 0:1]
    idx += 1
# data.head(50)
# print(x_dataset.shape)
print(x_dataset[1, 0, 0, :, 0])



(18, 500, 48, 20, 52)
(18, 500, 48, 1)
[0.25038 0.25109 0.25038 0.24977 0.29405 0.29303 0.24301 0.2409  0.29416
 0.29372 0.2348  0.2357  0.25649 0.25909 0.2908  0.29176 0.23719 0.23812
 0.25179 0.25299]


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

reshaped_x_dataset = x_dataset.reshape((-1, *x_dataset.shape[-2:]))
reshaped_y_dataset = y_dataset.reshape((-1, y_dataset.shape[-1]))

print(reshaped_x_dataset.shape)
print(reshaped_y_dataset.shape)

sc = StandardScaler()
enc = OneHotEncoder(sparse_output=False)
x_scaled = sc.fit_transform(reshaped_x_dataset.reshape(-1, reshaped_x_dataset.shape[-1])).reshape(reshaped_x_dataset.shape)
y_enc = enc.fit_transform(reshaped_y_dataset)

print(x_scaled.shape)
print(y_enc.shape)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_scaled.astype(np.float32), y_enc.astype(np.float32), train_size=0.2, random_state=42)


(432000, 20, 52)
(432000, 1)
(432000, 20, 52)
(432000, 18)


In [8]:
with open("dataset/dataset_pm.npy", "wb") as f:
    print(x_train.dtype)
    print(y_train.dtype)
    print(x_val.dtype)
    print(y_val.dtype)
    np.save(f, x_train)
    np.save(f, y_train)
    np.save(f, x_val)
    np.save(f, y_val)


float32
float32
float32
float32


In [10]:


def LSTM_model(X_train,y_train):
    # Define input layer
    input_layer = Input(shape=(X_train.shape[1],X_train.shape[2]))

    # Define encoder layers
    encoded = Bidirectional(LSTM(64,activation="tanh",return_sequences=True))(input_layer)
    encoded = LSTM(64, activation="tanh")(encoded)

    # Define decoder layers
    decoded = Dense(128, activation='selu')(encoded)
    decoded = Dropout(0.5)(decoded)
    decoded = Dense(y_train.shape[1], activation='softmax')(decoded)

    # Define LSTM model
    lstm_model = Model(inputs=input_layer, outputs=decoded)
    # Compile LSTM  model
    lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return lstm_model

def Transformer_model(x_train, y_train, num_layers, embedding_dim, num_heads):
    sequence_length = x_train.shape[-2]
    input_size = x_train.shape[-1]
    input_layer = Input(shape=(sequence_length,input_size))
    x_embedding = Dense(embedding_dim)(input_layer)
    x = x_embedding + keras_nlp.layers.SinePositionEncoding()(x_embedding)
    for i in range(num_layers):
        x = keras_nlp.layers.TransformerEncoder(embedding_dim, num_heads, 0.5)(x)
    print(x.shape)

# Define early stopping callback to monitor validation loss and stop if it doesn't improve for 5 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=5)
# Create model
# model = LSTM_model(x_train,y_train)
model = Transformer_model(x_train, y_train, num_layers=8, embedding_dim=512, num_heads=8)
# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model.fit(x_train, y_train, epochs=200, batch_size=256, validation_data=(x_val, y_val), callbacks=[early_stop])

# Plot the training history for loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

AttributeError: module 'keras_nlp.layers' has no attribute 'SinePositionEncoding'

In [13]:
from tensorflow.keras.layers import LayerNormalization, Layer, Dense, ReLU, Dropout
from keras.layers import MultiHeadAttention
from keras.layers import PositionEmbeddingFixedWeights



# Implementing the Add & Norm Layer
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer

    def call(self, x, sublayer_x):
        # The sublayer input and output need to be of the same shape to be summed
        add = x + sublayer_x

        # Apply layer normalization to the sum
        return self.layer_norm(add)

# Implementing the Feed-Forward Layer
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)  # First fully connected layer
        self.fully_connected2 = Dense(d_model)  # Second fully connected layer
        self.activation = ReLU()  # ReLU activation layer

    def call(self, x):
        # The input is passed into the two fully-connected layers, with a ReLU in between
        x_fc1 = self.fully_connected1(x)

        return self.fully_connected2(self.activation(x_fc1))

# Implementing the Encoder Layer
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()

    def call(self, x, padding_mask, training):
        # Multi-head attention layer
        multihead_output = self.multihead_attention(x, x, x, padding_mask)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        multihead_output = self.dropout1(multihead_output, training=training)

        # Followed by an Add & Norm layer
        addnorm_output = self.add_norm1(x, multihead_output)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(addnorm_output)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in another dropout layer
        feedforward_output = self.dropout2(feedforward_output, training=training)

        # Followed by another Add & Norm layer
        return self.add_norm2(addnorm_output, feedforward_output)

# Implementing the Encoder
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]

    def call(self, input_sentence, padding_mask, training):
        # Generate the positional encoding
        pos_encoding_output = self.pos_encoding(input_sentence)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        x = self.dropout(pos_encoding_output, training=training)

        # Pass on the positional encoded values to each encoder layer
        for i, layer in enumerate(self.encoder_layer):
            x = layer(x, padding_mask, training)

        return x

ImportError: cannot import name 'PositionEmbeddingFixedWeights' from 'keras.layers' (c:\Batu\Projects\fault-detection-kocsistem\.conda\Lib\site-packages\keras\api\layers\__init__.py)